In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mplfinance
!pip install efinance
!pip install yfinance

!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar xvzf ta-lib-0.4.0-src.tar.gz
import os
os.chdir('ta-lib')
!./configure --prefix=/usr
!make
!make install

os.chdir('../')
!pip install TA-Lib

In [ ]:
import pandas as pd
import numpy as np
import mplfinance as mpf
import matplotlib.pyplot as plt
import pandas as pd
import talib as ta
import yfinance as yf

In [ ]:
stock_codes1 = ["^GDAXI","^STOXX50E","^DJI","^GSPC","^IXIC","000001.SS","399001.SZ","^HSI","^N225"]
stock_codes1

# Data collection

In [ ]:
def calculate_ma(df, ma_periods=[5,10,20,30,60]):
    for period in ma_periods:
        df[f'{period}日均线'] = df['Close'].rolling(period).mean()
    return df

stock_codes = stock_codes1[0:] #data number
for each in stock_codes:
    data = yf.Ticker(each).history(start="2015-01-01", end="2019-12-31", interval="1d")
    df = data.loc[:, ['Open', 'High', 'Low', 'Close', 'Volume']]
    df = df.rename_axis('Date')
    # print(df)
    if len(df)==0:
        continue

    # MA
    df = calculate_ma(df)

    year = list(set(df.index.year))

    # Calculate MACD using talib
    df['macd'], df['macd_signal'], df['macd_hist'] = ta.MACD(df['Close'].values, fastperiod=12, slowperiod=26, signalperiod=9)

    for i in year:
        temp_y = df[df.index.year == i]
        month = list(set(temp_y.index.month))
        for j in month:
            temp_m = temp_y[temp_y.index.month == j]
            has_nan = temp_m.isna().any().any().any()
            if has_nan == 1:
                continue
            # MA chart
            ma_periods = [5, 10, 20, 30, 60]
            ma_plots = [mpf.make_addplot(temp_m[f'{period}日均线'], panel=0, ylabel='', title='') for period in ma_periods]
            # 假设 temp_m.index.min() 是一个 tz-aware Timestamp
            # 获取 temp_m.index.min() 的时区信息
            min_date = temp_m.index.min()
            min_date_tz = min_date.tzinfo
            start_date = pd.to_datetime("20140101").tz_localize(min_date_tz)

            if min_date > start_date:
                # MACD chart
                ap = []
                ap.append(mpf.make_addplot(temp_m['macd'], panel=2, color='fuchsia'))
                ap.append(mpf.make_addplot(temp_m['macd_signal'], panel=2, color='b'))
                colors = ['g' if v <= 0 else 'r' for v in temp_m["macd_hist"]]
                ap.append(mpf.make_addplot(temp_m['macd_hist'], panel=2, type='bar', color=colors))
                s = mpf.make_mpf_style(base_mpf_style='yahoo', mavcolors=['c', 'lime'])
                save_path = r'/content/drive/MyDrive/data521351/'
                filename = f"chart_{each}_{i}_{j}.png"
                mpf.plot(temp_m, addplot=ap+ma_plots, figratio=(40,20), panel_ratios=[2,2,2], type='candle', style=s, volume=True, volume_panel=1, savefig=save_path + filename,
                         ylabel='', ylabel_lower='', title='', )


# Label

In [ ]:
#生成label
import pandas as pd
import yfinance as yf
import os
import calendar
from datetime import datetime

# image_dir = "C:/Users/YANGJ/Downloads/Project小组/datagood/data_2000_3000"
# output_csv = "C:/Users/YANGJ/Downloads/Project小组/datatest/labels.csv"

image_dir = "/content/drive/MyDrive/data521/"
output_csv = "/content/drive/MyDrive/data521/labels.csv"

# Create an empty DataFrame to store the labels
labels_df = pd.DataFrame(columns=["Stock Code", "Year", "Month", "Label"])

# Iterate through the images in the directory
image_list = os.listdir(image_dir)
print(image_list)
for image_file in image_list:
    if image_file.endswith('.png'):
        # Extract stock code, year, and month from the image filename
        stock_code, year, month = image_file.split("_")[1:4]
        month = month.replace(".png", "")
         # Format the month string with leading zero if necessary
        month_str = month if len(month) == 2 else f"0{month}"

         # Construct the date range for fetching the stock data
        num_days = calendar.monthrange(int(year), int(month))[1]
        start_date = f"{year}{month_str}01"
        start_date = datetime.strptime(start_date, '%Y%m%d').strftime('%Y-%m-%d')
        end_date = f"{year}{month_str}{num_days}"
        end_date = datetime.strptime(end_date, '%Y%m%d').strftime('%Y-%m-%d')

        # Load the stock price data from Yahoo Finance
        ticker = yf.Ticker(stock_code)
        print(start_date)
        print(end_date)
        data = ticker.history(start=start_date, end=end_date)
        # print(data['Date '])
        # data['Date'] = pd.to_datetime(data['Date'])
        # data = data.set_index('Date')
        df = data.loc[:, ['Open', 'High', 'Low', 'Close', 'Volume']]
        df = df.rename_axis('Date')
        # df = df.rename(columns={'开盘': 'Open', '最高': 'High', '最低': 'Low', '收盘': 'Close', '成交量': 'Volume'})
        stock_data = df
        #stock_data = yf.download(stock_code, start=start_date, end=end_date)

        # Calculate the label based on the price change
        print(stock_data.iloc[0])
        first_price = stock_data.iloc[0]["Close"]
        last_price = stock_data.iloc[-1]["Close"]
        label1 = 1 if last_price > first_price else 0
        label2 = (last_price-first_price)/first_price

        # Append the label to the DataFrame
        labels_df = labels_df._append({"Stock Code": stock_code, "Year": year, "Month": month, "Return": label2, "Rise": label1}, ignore_index=True)

# Save the labels to a CSV file
labels_df.to_csv(output_csv, index=False)

print("Label generation complete!")